In [10]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [11]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [12]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://huggingface.co/blog/stable_diffusion")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents, OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x13ebdd3a0>)

In [13]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever, "Search_stable_diffusion", "Search instruction about how to use Stable Diffusion")

In [14]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool=ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [15]:
tools=[wiki,retriever_tool,arxiv_tool]

In [16]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

from react_template import get_react_prompt_template
prompt=get_react_prompt_template()


In [17]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm=llm,tools=tools,prompt=prompt)

In [23]:
from langchain.agents import AgentExecutor
agent_executer=AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executer.invoke({"input":"Tell me how Stable Difussion works", "tools":tools, "tool_names":"wikipedia, axiv and retriever"})



> Entering new AgentExecutor chain...

Invoking: `Search_stable_diffusion` with `{'query': 'How does Stable Diffusion work?'}`


Having seen the high-quality images that stable diffusion can produce, let's try to understand 
a bit better how the model functions.
Stable Diffusion is based on a particular type of diffusion model called Latent Diffusion, proposed in High-Resolution Image Synthesis with Latent Diffusion Models.
Generally speaking, diffusion models are machine learning systems that are trained to denoise random Gaussian noise step by step, to get to a sample of interest, such as an image. For a more detailed overview of how they work, check this colab.
Diffusion models have shown to achieve state-of-the-art results for generating image data. But one downside of diffusion models is that the reverse denoising process is slow because of its repeated, sequential nature. In addition, these models consume a lot of memory because they operate in pixel space, which becomes huge w

{'input': 'Tell me how Stable Difussion works',
 'tools': [WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/asakohayase/langchain_chatbot/venv/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
  Tool(name='Search_stable_diffusion', description='Search instruction about how to use Stable Diffusion', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x108f6c180>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x13ebdd3a0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x108f6c5e0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore